In [1]:
# Install required libraries
!pip install pandas numpy sqlalchemy bokeh pytest


# Define global paths

In [2]:
TRAIN_CSV = 'train.csv'
IDEAL_CSV = 'ideal.csv'
TEST_CSV = 'test.csv'
DB_PATH = 'results.db'
EXPORT_MAPPING_CSV = 'test_mapping.csv'
BOKEH_HTML = 'visualization.html'


# Import libraries

In [3]:
import os
import math
import logging
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Optional
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, HoverTool
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Custom Exceptions

In [4]:
class DataFormatError(Exception):
    """Raised when input CSV format is not as expected."""
    pass
class DatabaseError(Exception):
    """Raised for database-related errors."""
    pass

# Dataset Classes

In [5]:
class BaseDataset:
    def __init__(self, path: str):
        self.path = path
        self.df: Optional[pd.DataFrame] = None

    def load(self):
        try:
            self.df = pd.read_csv(self.path)
        except Exception as e:
            raise DataFormatError(f'Could not read CSV at {self.path}: {e}')
        return self.df
class TrainingDataset(BaseDataset):
    """
    Training dataset; expects X and 4 Y columns (Y1..Y4).
    Demonstrates inheritance from BaseDataset.
    """
    def validate(self):
        if self.df is None:
            raise DataFormatError('Training data not loaded yet.')
        cols = list(self.df.columns)
        if len(cols) < 5:
            raise DataFormatError('Training CSV must contain at least 5 columns: X and 4 Ys.')
        # Ensure first column is X
        # We'll accept columns starting with X or x
        if cols[0].lower() != 'x':
            logging.warning('First column not named X — treating first column as X anyway.')

    def get_x(self) -> np.ndarray:
        return self.df.iloc[:, 0].values

    def get_y_columns(self) -> List[str]:
        return list(self.df.columns[1:5])

    def get_training_series(self) -> Dict[str, pd.Series]:
        """Return mapping of training column name -> series"""
        cols = self.get_y_columns()
        return {c: self.df[c] for c in cols}


class IdealFunctions(BaseDataset):
    """Loads ideal functions CSV (X + 50 columns)."""
    def validate(self):
        if self.df is None:
            raise DataFormatError('Ideal functions not loaded yet.')
        if self.df.shape[1] < 2:
            raise DataFormatError('Ideal CSV must contain X and at least one ideal function column.')

    def get_x(self) -> np.ndarray:
        return self.df.iloc[:, 0].values

    def get_function_columns(self) -> List[str]:
        return list(self.df.columns[1:])

    def get_function_series(self, name: str) -> pd.Series:
        return self.df[name]


class TestDataset(BaseDataset):
    """Loads test CSV with X,Y pairs."""
    def validate(self):
        if self.df is None:
            raise DataFormatError('Test data not loaded yet.')
        if self.df.shape[1] < 2:
            raise DataFormatError('Test CSV must contain X and Y columns.')

    def get_pairs(self) -> List[Tuple[float, float]]:
        xs = self.df.iloc[:, 0].values
        ys = self.df.iloc[:, 1].values
        return list(zip(xs, ys))


# Database Manager

In [6]:
# Database Manager
class DatabaseManager:
    """Handles SQLite DB creation and table writes using sqlalchemy."""

    def __init__(self, db_path: str = DB_PATH):
        self.db_path = db_path
        self.engine = None

    def connect(self):
        try:
            self.engine = create_engine(f'sqlite:///{self.db_path}')
            logging.info(f'Connected to SQLite DB at {self.db_path}')
        except SQLAlchemyError as e:
            raise DatabaseError(f'Could not create DB engine: {e}')

    def write_dataframe(self, df: pd.DataFrame, table_name: str, if_exists='replace'):
        if self.engine is None:
            self.connect()
        try:
            df.to_sql(table_name, con=self.engine, index=False, if_exists=if_exists)
            logging.info(f'Wrote table {table_name} (rows: {len(df)})')
        except Exception as e:
            raise DatabaseError(f'Failed to write table {table_name}: {e}')


# Core Mapping Logic

In [7]:
class Mapper:
    def __init__(self, training: TrainingDataset, ideal: IdealFunctions):
        self.training = training
        self.ideal = ideal
        # Will contain mapping from training col -> chosen ideal col
        self.chosen_map: Dict[str, str] = {}
        # Largest deviations per training col
        self.max_training_deviation: Dict[str, float] = {}

    def choose_best_ideal_for_each_training(self):
        """For each training Y column, find the ideal function column minimizing sum of squared deviations."""
        x_train = self.training.get_x()
        ideal_x = self.ideal.get_x()
        # Basic check: x grids should align. If not, align by index.
        if not np.array_equal(x_train, ideal_x):
            logging.warning('X-values of training and ideal functions do not match exactly. Aligning by index.')

        ideal_cols = self.ideal.get_function_columns()
        for tcol in self.training.get_y_columns():
            y_train = self.training.df[tcol].values
            best_col = None
            best_ssq = float('inf')
            best_residuals = None

            for icol in ideal_cols:
                y_ideal = self.ideal.df[icol].values
                # resize if lengths differ
                n = min(len(y_train), len(y_ideal))
                resid = y_train[:n] - y_ideal[:n]
                ssq = np.sum(resid ** 2)
                if ssq < best_ssq:
                    best_ssq = ssq
                    best_col = icol
                    best_residuals = resid

            self.chosen_map[tcol] = best_col
            # largest absolute deviation
            self.max_training_deviation[tcol] = float(np.max(np.abs(best_residuals)))
            logging.info(
                f'Training {tcol} matched to ideal {best_col} '
                f'with max dev {self.max_training_deviation[tcol]:.6f}'
            )
        return self.chosen_map

    def map_test_points(self, test_pairs: List[Tuple[float, float]]) -> pd.DataFrame:
        """
        Map each test point to one of the chosen ideal functions if within threshold.
        Threshold per training = max_training_deviation * sqrt(2).
        Returns DataFrame with columns: X, Y, DeltaY, AssignedFunction (or None)
        """
        rows = []
        # Build reverse map: ideal col -> training col
        reverse_map = {v: k for k, v in self.chosen_map.items()}

        for x_val, y_val in test_pairs:
            best_ideal = None
            best_delta = float('inf')
            # For each chosen ideal (4 of them)
            for tcol, icol in self.chosen_map.items():
                ideal_df = self.ideal.df
                try:
                    y_ideal = np.interp(x_val, ideal_df.iloc[:, 0].values, ideal_df[icol].values)
                except Exception:
                    y_ideal = float('nan')
                delta = abs(y_val - y_ideal)
                # threshold
                threshold = self.max_training_deviation[tcol] * math.sqrt(2)
                if delta <= threshold and delta < best_delta:
                    best_delta = delta
                    best_ideal = icol
            rows.append({
                'X': x_val,
                'Y': y_val,
                'DeltaY': (best_delta if best_ideal is not None else None),
                'IdealFunction': (best_ideal if best_ideal is not None else None)
            })

        df = pd.DataFrame(rows)
        return df


# Visualization

In [8]:
class Visualizer:
    """Creates a Bokeh plot showing training data, chosen ideal functions, and mapped test points."""

    def __init__(self, training: TrainingDataset, ideal: IdealFunctions, test_df: pd.DataFrame, mapper: Mapper):
        self.training = training
        self.ideal = ideal
        self.test_df = test_df
        self.mapper = mapper

    def create_plot(self, out_path: str = BOKEH_HTML):
        output_file(out_path, title='Training, Ideal Functions and Test Mapping')
        p = figure(
            title='Training vs Chosen Ideal Functions and Test Points',
            x_axis_label='X',
            y_axis_label='Y',
            width=1000,
            height=600,
            tools='pan,wheel_zoom,box_zoom,reset,save'
        )

        # Plot training series
        x = self.training.get_x()
        for tcol in self.training.get_y_columns():
            p.line(x, self.training.df[tcol].values, legend_label=f'Train {tcol}', line_width=2)

        # Plot chosen ideal functions
        for tcol, icol in self.mapper.chosen_map.items():
            ideal_x = self.ideal.get_x()
            p.line(
                ideal_x,
                self.ideal.df[icol].values,
                legend_label=f'Ideal {icol} (for {tcol})',
                line_dash='dashed'
            )

        # Plot test points, colored by assignment
        assigned = self.test_df.dropna(subset=['IdealFunction'])
        unassigned = self.test_df[self.test_df['IdealFunction'].isna()]

        if not assigned.empty:
            src_assigned = ColumnDataSource(assigned)
            p.circle('X', 'Y', source=src_assigned, size=8, legend_label='Assigned Test Points')

        if not unassigned.empty:
            src_un = ColumnDataSource(unassigned)
            p.cross('X', 'Y', source=src_un, size=8, legend_label='Unassigned Test Points')

        hover = HoverTool(tooltips=[('X', '@X'), ('Y', '@Y'), ('DeltaY', '@DeltaY'), ('IdealFunction', '@IdealFunction')])
        p.add_tools(hover)

        p.legend.location = 'top_left'
        save(p)
        logging.info(f'Bokeh visualization saved to {out_path}')


# Utilities and Unit Tests

In [9]:
def run_pipeline(train_csv: str = TRAIN_CSV, ideal_csv: str = IDEAL_CSV, test_csv: str = TEST_CSV):
    # Load training data
    training = TrainingDataset(train_csv)
    training.load()
    training.validate()

    # Load ideal functions
    ideal = IdealFunctions(ideal_csv)
    ideal.load()
    ideal.validate()

    # Load test data
    test = TestDataset(test_csv)
    test.load()
    test.validate()

    # DB manager
    db = DatabaseManager()
    db.connect()
    # write raw tables
    db.write_dataframe(training.df, 'training')
    db.write_dataframe(ideal.df, 'ideal_functions')
    db.write_dataframe(test.df, 'test_raw')

    # Mapper
    mapper = Mapper(training, ideal)
    mapper.choose_best_ideal_for_each_training()
    test_pairs = test.get_pairs()
    mapping_df = mapper.map_test_points(test_pairs)

    # Save mapping to DB and CSV
    db.write_dataframe(mapping_df, 'test_mapping')
    mapping_df.to_csv(EXPORT_MAPPING_CSV, index=False)
    logging.info(f'Mapping exported to {EXPORT_MAPPING_CSV}')

    # Visualization
    viz = Visualizer(training, ideal, mapping_df, mapper)
    viz.create_plot(BOKEH_HTML)

    return {
        'db_path': db.db_path,
        'mapping_csv': EXPORT_MAPPING_CSV,
        'bokeh_html': BOKEH_HTML,
    }


# Unit tests using assertions

In [10]:


def _unit_tests():
    logging.info('Running basic unit tests...')

    #  Check files exist
    for p in [TRAIN_CSV, IDEAL_CSV, TEST_CSV]:
        assert os.path.exists(p), f'Expected file not found: {p}'

    #  Load and basic checks
    tr = TrainingDataset(TRAIN_CSV)
    tr.load()
    tr.validate()

    idf = IdealFunctions(IDEAL_CSV)
    idf.load()
    idf.validate()

    te = TestDataset(TEST_CSV)
    te.load()
    te.validate()

    #  Mapper choose logic
    m = Mapper(tr, idf)
    chosen = m.choose_best_ideal_for_each_training()
    assert len(chosen) >= 1, 'No chosen mappings found.'

    # Map test points
    mapping_df = m.map_test_points(te.get_pairs())
    assert 'IdealFunction' in mapping_df.columns, 'Mapping result missing IdealFunction column.'

    logging.info('Unit tests passed (basic).')


In [11]:
if __name__ == '__main__':
    try:
        _unit_tests()
        results = run_pipeline()
        logging.info('Pipeline completed successfully. Outputs:')
        for k, v in results.items():
            logging.info(f' - {k}: {v}')
    except AssertionError as ae:
        logging.error(f'AssertionError during tests/pipeline: {ae}')
    except DataFormatError as dfe:
        logging.error(f'Data format problem: {dfe}')
    except DatabaseError as dbe:
        logging.error(f'Database problem: {dbe}')
    except Exception as e:
        logging.exception(f'Unhandled exception: {e}')

In [12]:
import sqlite3
import pandas as pd

# Connect to the DB
conn = sqlite3.connect("results.db")

# Check which tables exist
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:")
print(tables)

Tables in DB:
              name
0         training
1  ideal_functions
2         test_raw
3     test_mapping
